# Multiscale MILP (using Attributes)

This is a continuation of 'One Location, One Temporal Scale, One Operation, Linear Programming Example' [Example 1]. Refer Example 1 to learn the basics on how Energia models processes. 

In this example, we add another Process [Solar PV] and Storage [Li-ion Battery]. Technology choice is modeled using binaries. Moreover, the model is multiscale as the operational capacities are decision variables. 

In [1]:
from energia import *

m = Model()
m.q = Periods()
m.y = 4 * m.q
m.usd = Currency()

## Resources

### Set bounds on Resource flows

Unlike wind which has bound on the total consumption, we set a daily limit on solar energy. The same bound is repeated in each quarter. The following constraints are written.

$\mathbf{cons}_{solar, network, quarter_0} \leq 100$

$\mathbf{cons}_{solar, network, quarter_1} \leq 100$

$\mathbf{cons}_{solar, network, quarter_2} \leq 100$

$\mathbf{cons}_{solar, network, quarter_3} \leq 100$

In [2]:
m.solar = Resource(consume_max=[100] * 4)
m.wind = Resource(consume_max=100 * 4)
m.power = Resource(demand_nominal=180, demand_min=[0.6, 0.7, 0.8, 0.3])

2025-10-22 14:38:49,437 [INFO] Balance for solar in (l0, q): initializing
2025-10-22 14:38:49,438 [INFO] ✔ Completed in 0.00019049644470214844 seconds
2025-10-22 14:38:49,438 [INFO] Binding consume in domain (solar, l0, q)
2025-10-22 14:38:49,438 [INFO] ✔ Completed in 0.00012540817260742188 seconds
2025-10-22 14:38:49,439 [INFO] Balance for wind in (l0, y): initializing
2025-10-22 14:38:49,439 [INFO] ✔ Completed in 9.560585021972656e-05 seconds
2025-10-22 14:38:49,439 [INFO] Binding consume in domain (wind, l0, y)
2025-10-22 14:38:49,440 [INFO] ✔ Completed in 6.508827209472656e-05 seconds
2025-10-22 14:38:49,440 [INFO] Balance for power in (l0, q): initializing
2025-10-22 14:38:49,441 [INFO] ✔ Completed in 0.00011801719665527344 seconds
2025-10-22 14:38:49,441 [INFO] Binding release in domain (power, l0, q)
2025-10-22 14:38:49,442 [INFO] ✔ Completed in 0.00014662742614746094 seconds


## Operations 

### Capacity as a variable 

Here we want the optimization problem to determine the optimal capacity. Moreover, we set binaries to avoid the lower bound being adhered to if the process is not set up. 

If the bounds are meant to be compulsory limits, skip the .x 

In [3]:
m.wf = Process(
    m.power == -1.0 * m.wind,
    capacity_max=100,
    capacity_min=10,
    capacity_optional=True,
    operate_normalize=True,
    operate_max=[0.9, 0.8, 0.5, 0.7],
    capex=990637 + 3354,
    opex=49,
)

2025-10-22 14:38:49,446 [INFO] Binding capacity in domain (wf, l0, y)
2025-10-22 14:38:49,447 [INFO] ✔ Completed in 0.00018453598022460938 seconds
2025-10-22 14:38:49,447 [INFO] Binding capacity in domain (wf, l0, y)
2025-10-22 14:38:49,447 [INFO] ✔ Completed in 0.0001552104949951172 seconds
2025-10-22 14:38:49,448 [INFO] Binding operate in domain (wf, l0, q)
2025-10-22 14:38:49,448 [INFO] ✔ Completed in 0.000339508056640625 seconds
2025-10-22 14:38:49,449 [INFO] Mapping operate: (wf, l0, q) → (wf, l0, y)
2025-10-22 14:38:49,450 [INFO] ✔ Completed in 9.179115295410156e-05 seconds


Unlike in Example 1, where the capacity was know, capacity is a variable here. 

Moreover, the expenditure associated with operating and capacitating are different

In [4]:
m.pv = Process(
    m.power == -1 * m.solar,
    capacity_max=100,
    capacity_min=10,
    capacity_optional=True,
    operate_normalize=True,
    operate_max=[0.6, 0.8, 0.9, 0.7],
    capex=567000 + 872046,
    opex=90000,
)

2025-10-22 14:38:49,455 [INFO] Binding capacity in domain (pv, l0, y)
2025-10-22 14:38:49,456 [INFO] ✔ Completed in 0.00018835067749023438 seconds
2025-10-22 14:38:49,456 [INFO] Binding capacity in domain (pv, l0, y)
2025-10-22 14:38:49,457 [INFO] ✔ Completed in 0.00016832351684570312 seconds
2025-10-22 14:38:49,457 [INFO] Binding operate in domain (pv, l0, q)
2025-10-22 14:38:49,457 [INFO] ✔ Completed in 0.00019598007202148438 seconds
2025-10-22 14:38:49,458 [INFO] Mapping operate: (pv, l0, q) → (pv, l0, y)
2025-10-22 14:38:49,459 [INFO] ✔ Completed in 0.00024080276489257812 seconds


### Storage Operation

energia now allows storing to require the use of other resources, example power for hydrogen cryogenic storage. 

Provide an equation similar to Process, in this case the basis is the stored resource 
If no other resource is provided, it is assumed to be the charging/discharging efficiency

Note that the following are created internally: 
1. auxilary resource  with name resource.stored 
2. charging and discharging processes as storage.charge and storage.discharge 

The parameters for each of these can be set individually, thus allowing for a wide range of modeling approaches 

In [5]:
m.lii = Storage(
    m.power == 0.9,
    capacity_max=100,
    capacity_min=10,
    capacity_optional=True,
    charge_capacity_max=100,
    discharge_capacity_max=100,
    capex=1302182 + 41432,
    inventory_cost=2000,
)

2025-10-22 14:38:49,464 [INFO] Binding capacity in domain (lii.charge, l0, y)
2025-10-22 14:38:49,464 [INFO] ✔ Completed in 9.870529174804688e-05 seconds
2025-10-22 14:38:49,465 [INFO] Binding capacity in domain (lii.discharge, l0, y)
2025-10-22 14:38:49,465 [INFO] ✔ Completed in 7.224082946777344e-05 seconds
2025-10-22 14:38:49,466 [INFO] Binding invcapacity in domain (lii.stored, l0, y)
2025-10-22 14:38:49,466 [INFO] ✔ Completed in 0.000156402587890625 seconds
2025-10-22 14:38:49,467 [INFO] Binding invcapacity in domain (lii.stored, l0, y)
2025-10-22 14:38:49,467 [INFO] ✔ Completed in 0.00014448165893554688 seconds
2025-10-22 14:38:49,468 [INFO] Balance for lii.stored in (l0, y): initializing


## Locating Operations

Operations can be located as 

operation.locate(\<list of locations\>)

or 

m.location.operations(\<list of operations\>)

They both do the same thing 

In [6]:
m.locate(m.wf, m.pv, m.lii)

2025-10-22 14:38:49,473 [INFO] Balance for power in (l0, q): adding produce(power, l0, q, operate, wf)
2025-10-22 14:38:49,473 [INFO] ✔ Completed in 0.00014781951904296875 seconds
2025-10-22 14:38:49,474 [INFO] Balance for wind in (l0, y): adding expend(wind, l0, y, operate, wf)
2025-10-22 14:38:49,475 [INFO] ✔ Completed in 0.00011014938354492188 seconds
2025-10-22 14:38:49,476 [INFO] Balance for power in (l0, q): adding produce(power, l0, q, operate, pv)
2025-10-22 14:38:49,476 [INFO] ✔ Completed in 0.00013637542724609375 seconds
2025-10-22 14:38:49,477 [INFO] Balance for solar in (l0, q): adding expend(solar, l0, q, operate, pv)
2025-10-22 14:38:49,478 [INFO] ✔ Completed in 0.00015306472778320312 seconds
2025-10-22 14:38:49,478 [INFO] Assuming  lii.stored inventory capacity is unbounded in (l0, y)
2025-10-22 14:38:49,478 [INFO] Assuming inventory of lii.stored is unbounded in (l0, y)
2025-10-22 14:38:49,478 [INFO] Assuming inventory of lii.stored is bound by inventory capacity in (l0

## Inventory Balance

Inventory is passed on from one time period (t - 1) to the next (t) and hence features in the general resource balance for resource.stored 

## Optimize!

In [7]:
m.usd.spend.opt()

2025-10-22 14:38:49,494 [INFO] Mapping spend: (usd, l0, y, capacity, wf) → (usd, l0, y)
2025-10-22 14:38:49,494 [INFO] ✔ Completed in 0.00011372566223144531 seconds
2025-10-22 14:38:49,495 [INFO] Mapping spend: (usd, l0, y, operate, wf) → (usd, l0, y)
2025-10-22 14:38:49,495 [INFO] ✔ Completed in 8.511543273925781e-05 seconds
2025-10-22 14:38:49,495 [INFO] Mapping spend: (usd, l0, y, capacity, pv) → (usd, l0, y)
2025-10-22 14:38:49,496 [INFO] ✔ Completed in 7.724761962890625e-05 seconds
2025-10-22 14:38:49,496 [INFO] Mapping spend: (usd, l0, y, operate, pv) → (usd, l0, y)
2025-10-22 14:38:49,497 [INFO] ✔ Completed in 0.00011658668518066406 seconds
2025-10-22 14:38:49,497 [INFO] Mapping spend: (usd, l0, y, invcapacity, lii.stored) → (usd, l0, y)
2025-10-22 14:38:49,498 [INFO] ✔ Completed in 0.00010991096496582031 seconds
2025-10-22 14:38:49,498 [INFO] Mapping spend: (usd, l0, y, inventory, lii.stored) → (usd, l0, y)
2025-10-22 14:38:49,498 [INFO] ✔ Completed in 0.00010991096496582031 se

Set parameter Username
Academic license - for non-commercial use only - expires 2026-08-01
Read MPS format model from file Program(m).mps
Reading time = 0.00 seconds
PROGRAM(M): 85 rows, 78 columns, 198 nonzeros


2025-10-22 14:38:49,509 [INFO] Optimizing Program(m) using gurobi


Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13700, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 85 rows, 78 columns and 198 nonzeros
Model fingerprint: 0x239d525a
Variable types: 75 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [6e-01, 1e+06]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [7e+01, 4e+02]
Presolve removed 70 rows and 63 columns
Presolve time: 0.00s
Presolved: 15 rows, 15 columns, 44 nonzeros
Variable types: 15 continuous, 0 integer (0 binary)

Root relaxation: objective 3.006497e+08, 4 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    3.006497e+08 3.0065e+08  0.0

2025-10-22 14:38:49,519 [INFO] Solution found. Use .output() to display it
2025-10-22 14:38:49,520 [INFO] Creating Solution object, check.solution


In [8]:
m.show(True)

# Mathematical Program for Program(m)

<br><br>

## Index Sets

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

###  Functions

<IPython.core.display.Math object>

## Solution

### Inventory Profiles

The inventory maintained in each time period is:

In [9]:
m.inventory.output()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

The amount charged into inventory is:

In [10]:
m.produce(m.power.lii, m.lii.charge.operate, m.q).output()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

The amount discharged from inventory is:

In [11]:
m.produce(m.power, m.lii.discharge.operate, m.q).output()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

### Integer Decisions 

All the operations are setup in this case

In [12]:
m.capacity.reporting.output()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [13]:
m.capacity.output()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>